In [ ]:
import requests
import json
import time
import pandas as pd

vacancy = input('Напишите наименование вакансии:')
vacanciesDescription = []
result = dict()

def getPage(vacancy, page = 0):
    """
    Функция для получения страницы со списком вакансий.
    Аргументы:
        page - индекс страницы, значения от 0 до 20. Значение по умолчанию 0, т.е. первая страница выдачи поиска
        name - наименование вакансии
    """
    
    #Словарь с параметрами запроса
    
    params = {
        'text': 'NAME:{}'.format(vacancy), # текст фильтра с наименованием вакансии
        'area': 113, # поиск ощуществляется по вакансиям России, 113 - значение из справочника регионов hh.ru 
        'page': page, # индекс страницы поиска на hh.ru
        'per_page': 100 # кол-во вакансий на одной странице, 100 - максимальное значение
    }
    
    
    req = requests.get('https://api.hh.ru/vacancies', params)
    data = req.content.decode() # декодирование, чтобы кириллица отображалась корректно
    req.close()
    return json.loads(data)

#Т.к. в списке вакансий нет полного описания вакансии с перечисленными ключевычми навыками, необходимо пройти по всем [items] из выдачи, для этого функция: 

def getVacancy(item):
    """
    Функция для получения полного описания вакансии
    """
    req = requests.get(item['url'])
    data = req.content.decode()
    data = json.loads(data)
    return data


for page in range(0, 20): #20 т.к. максимально возможная выдача 20 страниц и 2000 вакансий
    reqResult = getPage(vacancy, page)
    vacanciesDescription.extend(reqResult['items'])
    if (reqResult['pages'] - page) <= 1: #перервывает цикл, если результатов менее 2000
        break
    time.sleep(0.25) #небольшая задержка, чтобы не перегрузить hh
    

for item in vacanciesDescription:
    vacancyDescription = getVacancy(item)
    for skill in vacancyDescription['key_skills']:
        if skill['name'] in result:
            result[skill['name']] =  result[skill['name']] + 1 
        else: result[skill['name']] = 1
    time.sleep(0.25) #небольшая задержка, чтобы не перегрузить hh

df = pd.DataFrame(data=result.items(), columns=['Навык', 'Количество упоминаний'])
df.sort_values(by='Количество упоминаний', ascending=False).reset_index(drop=True).to_excel('результат.xlsx')